In [1]:
import pandas as pd

In [2]:
first_df = pd.read_csv("firstfile.csv")
first_df.head()

,amount
0,1000.00
1,200.00
2,555.12
3,876.34
4,345.00


In [3]:
first_file_sum = first_df['amount'].sum()
print(first_file_sum)

2976.46


In [4]:
second_df = pd.read_csv("secondfile.csv")
second_file_sum = second_df['amount'].sum()
print(second_file_sum)

3256


In [5]:
third_df = pd.read_csv("thirdfile.csv")
third_file_sum = third_df['amount'].sum()
print(third_file_sum)

3208


In [6]:
import openpyxl

In [7]:
! type parameterfile.txt

[global]
some value
$$PROCESS_DT = 202204
new line


In [8]:
# Code to fetch process date
param_file = open('parameterfile.txt')
process_dt = "".join([line.strip()[-6:] for line in param_file if '$$PROCESS_DT' in line])
print(process_dt)

202204


In [9]:
# Code for excel file name for respective month
import calendar
month_abbreviation = calendar.month_abbr[int(str(process_dt)[-2:])] # This line will convert month number to month abbreviation
target_file_name = 'Target File - ' + month_abbreviation + str(process_dt)[0:4] + '.xlsx'
print(target_file_name)

Target File - Apr2022.xlsx


In [10]:
import pymysql
try:
    connection = pymysql.connect(host = "localhost",
                     user = "sai",
                     password = "*******",
                     db = "dummydb")
    cursor = connection.cursor()
    first_sq_query = '''SELECT SUM(AMOUNT) FROM DUMMYTABLE \
    WHERE PROCESS_DT = %d''' %(int(process_dt))
    cursor.execute(first_sq_query)
    first_sq_sum = cursor.fetchone()[0]
    second_sq_query = '''SELECT MIN(AMOUNT) FROM DUMMYTABLE \
    WHERE PROCESS_DT = %d''' %(int(process_dt))
    cursor.execute(second_sq_query)
    second_sq_sum = cursor.fetchone()[0]
    connection.commit()
except Exception as e:
    print("something went wrong : ",e)
finally:
    cursor.close()
    connection.close()

In [11]:
#Code to create excel file
wb = openpyxl.Workbook()
wb.save(target_file_name)

In [23]:
wb1 = openpyxl.load_workbook(target_file_name)
ws1 = wb1.active
ws1.merge_cells('A1:J1')
yellow = openpyxl.styles.colors.Color(rgb='FFFF00')
filling = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=yellow)
ws1['A1'].fill = filling
ws1['A1'] = month_abbreviation + ' - ' + str(process_dt)[:4]
ws1.cell(1,1).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
ws1['G2'] = 'FIRST FILE SUM'
ws1['H2'] = '$' + str(first_file_sum)
ws1['G3'] = 'SECOND FILE SUM'
ws1['H3'] = '$' + str(second_file_sum)
ws1['G4'] = 'THIRD FILE SUM'
ws1['H4'] = '$' + str(third_file_sum)
three_file_diff = first_file_sum + second_file_sum + third_file_sum
ws1['H5'] = '$' + str(three_file_diff)
ws1['A2'] = 'FIRST QUERY SUM'
ws1['B2'] = '$' + str(first_sq_sum)
ws1['A3'] = 'SECOND QUERY SUM'
ws1['B3'] = '$' + str(second_sq_sum)
ws1['A4'] = 'FIRST AND SECOND QUERY DIFF'
two_sq_diff = first_sq_sum - second_sq_sum
ws1['B4'] = '$' + str(two_sq_diff)
thin = openpyxl.styles.Side(border_style="thin", color='00000000')
for column in 'ABCDEFGHIJ':
    for row in range(1,6):
        cell_number = str(column) + str(row)
        ws1[cell_number].border = openpyxl.styles.Border(top=thin, left=thin, right=thin, bottom=thin)
ws1['C7'] = 'DIFFERENCE'
ws1['C7'].font = openpyxl.styles.Font(color='00FF0000')
ws1['D7'] = '$ (' + str(round(float(three_file_diff) - float(two_sq_diff),2)) + ')'
ws1['D7'].font = openpyxl.styles.Font(color='00FF0000')
wb1.save(target_file_name)